In [10]:
import cv2

In [11]:
import numpy as np

In [12]:
image = cv2.imread('SudokuTest2.jpg')

In [13]:
def process_raw(image):
    gray_img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blur_gry=cv2.GaussianBlur(gray_img,(3,3),1)
    _, th3 = cv2.threshold(blur_gry, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    ret,thresh2 = cv2.threshold(th3, 127, 255, cv2.THRESH_BINARY_INV)

    return thresh2

In [14]:
def return_contours(processed):
    contours, hierarchy = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for i in range(len(sorted_contours)):
        approx = cv2.approxPolyDP(sorted_contours[i], 0.01*cv2.arcLength(sorted_contours[i],True),True)
        if len(approx) == 4:
            return approx;
    return [1]

In [15]:
def sort_approx(approx):
    sorted_approx = sorted(approx,key=lambda k:[k[0][1],k[0][0]])
    sorted_approx[:2]= sorted(approx[0:2],key=lambda k:k[0][0])
    sorted_approx[2:4]= sorted(approx[2:4],key=lambda k:k[0][0])
    sorted_points=[] #pehele hi lst m convert krne hai !!
    for i in range(4):
        sorted_points.append(sorted_approx[i].tolist()[0])
    return np.float32(sorted_points)

In [16]:
def Extract_sudoko(sorted_points,image):
    points_B = np.float32([[0,0], [450,0], [0,450], [450,450]])
    M = cv2.getPerspectiveTransform(sorted_points, points_B)
    warped = cv2.warpPerspective(image, M, (450,450))
    return warped

In [17]:
def show_frames(image):    
    processed_img=process_raw(image)

    approx = return_contours(processed_img)

    Sudoku_grid= np.zeros((450,450))

    if len(approx)==4:
        sorted_points = sort_approx(approx)
        cv2.drawContours(image, [approx], 0, (0,0,0), 5)
        Sudoku_grid= Extract_sudoko(sorted_points,image)

    return image,Sudoku_grid


In [9]:
cap = cv2.VideoCapture(0)

while (cap.isOpened()):
    ret,frame=cap.read()
    if ret == True:
        img , grid = show_frames(frame)
        cv2.imshow("feed",img)
        cv2.imshow("Sudoko",grid)
        if cv2.waitKey(1) & 0xFF ==ord("q"):
            break

cap.release()
cv2.destroyAllWindows()

In [18]:
img , grid = show_frames(image)
cv2.imshow("feed",img)
cv2.imshow("Sudoko",grid)
cv2.waitKey(0)
cv2.imwrite('TestGrid2.jpg', grid)
cv2.destroyAllWindows()